# Running training in 2019 and 2020

> [https://github.com/BMClab/covid19](https://github.com/BMClab/covid19)  
> [Laboratory of Biomechanics and Motor Control](http://pesquisa.ufabc.edu.br/bmclab/)  
> Federal University of ABC, Brazil

<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Environment" data-toc-modified-id="Environment-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Environment</a></span></li><li><span><a href="#Helping-functions" data-toc-modified-id="Helping-functions-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Helping functions</a></span></li></ul></li><li><span><a href="#Load-dataset" data-toc-modified-id="Load-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load dataset</a></span><ul class="toc-item"><li><span><a href="#Basic-information-about-the-dataset" data-toc-modified-id="Basic-information-about-the-dataset-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Basic information about the dataset</a></span></li></ul></li><li><span><a href="#Load-dataset-with-indexes-of-policy-responses-to-the-coronavirus-pandemic" data-toc-modified-id="Load-dataset-with-indexes-of-policy-responses-to-the-coronavirus-pandemic-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load dataset with indexes of policy responses to the coronavirus pandemic</a></span></li><li><span><a href="#Running-activity" data-toc-modified-id="Running-activity-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running activity</a></span><ul class="toc-item"><li><span><a href="#Number-of-running-activities-per-period" data-toc-modified-id="Number-of-running-activities-per-period-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Number of running activities per period</a></span></li></ul></li><li><span><a href="#Data-distribution" data-toc-modified-id="Data-distribution-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Data distribution</a></span><ul class="toc-item"><li><span><a href="#Difference-between-2020-and-2019" data-toc-modified-id="Difference-between-2020-and-2019-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Difference between 2020 and 2019</a></span></li><li><span><a href="#Histrogram" data-toc-modified-id="Histrogram-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Histrogram</a></span></li></ul></li><li><span><a href="#Volume-of-training:-average-and-95%-confidence-interval" data-toc-modified-id="Volume-of-training:-average-and-95%-confidence-interval-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Volume of training: average and 95% confidence interval</a></span><ul class="toc-item"><li><span><a href="#Calculus-of-central-tendency-and-confidence-interval-for-each-year" data-toc-modified-id="Calculus-of-central-tendency-and-confidence-interval-for-each-year-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Calculus of central tendency and confidence interval for each year</a></span></li><li><span><a href="#Calculate-differences-between-years" data-toc-modified-id="Calculate-differences-between-years-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Calculate differences between years</a></span><ul class="toc-item"><li><span><a href="#Statistical-inference" data-toc-modified-id="Statistical-inference-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Statistical inference</a></span></li></ul></li><li><span><a href="#Plots" data-toc-modified-id="Plots-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Plots</a></span><ul class="toc-item"><li><span><a href="#Infographic" data-toc-modified-id="Infographic-6.3.1"><span class="toc-item-num">6.3.1&nbsp;&nbsp;</span>Infographic</a></span></li></ul></li></ul></li></ul></div>

## Setup

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tqdm
from spm1d.stats import ttest_paired, ttest2, _spm
import matplotlib.patches as patches
%load_ext watermark  

%watermark
%watermark --iversions

### Environment

In [ ]:
path2 = r'./../data/'

pd.set_option('display.float_format', lambda x: '%.4g' % x)
plt.rcParams.update({'font.size': 14, 'xtick.labelsize': 12, 'ytick.labelsize': 12})

sns.set_style('whitegrid', rc={'xtick.bottom': True, 'xtick.top': True, 'ytick.left': True,
              'ytick.right': True, 'xtick.direction': 'in', 'ytick.direction': 'in'})

# plot colors
colors = sns.color_palette()
display(colors)
c1 = colors[0]
c2 = colors[3]
c3 = 'k'
c4 = colors[2]
c5 = colors[1]
# feature (dependent variable)
variables, units = ['distance', 'duration'], ['km', 'min']
freq = 'w'
freq_s = 'week'
# confidence interval
estimate = np.mean
level = 99
method = 'normal'
method = 'bootstrap'  # use to generate the final figures in paper, but much slower
# Statistics
alpha = 0.01
two_tailed = True
spmtest = ttest_paired
#spmtest = wilcoxon
# other variables:
ages = ['18 - 34', '35 - 54', '55 +']
genders = ['F', 'M']
years = ['2019', '2020']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']
freqs = ['d', 'w', 'm']
freqs_s = ['day', 'week', 'month']

### Helping functions

In [ ]:
def ci_calc(y, estimate=np.nanmean, level=95, axis=0, method='bootstrap',
            n_boot=1000, n=None, seed=None):
    """Calculate `level`% confidence interval of the `estimate` for `y`.

    Parameters
    ----------
    y : 1-d or 2-d array_like
        Calculate the confidence interval of these values.
    estimate : numpy function (optional, default=np.nanmean)
        Function (unknown parameter) to which the confidence interval will be
        estimated
    level : number (optional, default=95)
        Confidence level (in %, in the interval ]0, 100[) of the confidence
        interval
    axis : int (optional, default=0)
        Axis of y which the confidence interval will be estimated
    method: string (optional, default=`'bootstrap'`)
        Valid options: `'bootstrap'`, `'bootstrap_sns'` (Seaborn), `'normal'`.
        `'normal'` method calculates confidence interval for the `'estimate'`
        of `y` assuming normal distribution and unknown mean and unknown
        standard deviation.
        `'bootstrap'` and `'bootstrap_sns'` methods calculate confidence
        interval for the `estimate` of `y` by percentile bootstrap.
    n_boot : int (optional, default=1000)
        Number of bootstrap iteractions
    n : {None, int} (optional, default=None)
        Number of samples to bootstrap (only works with method `'bootstrap'`)
    seed : {None, int, ..., Generator} (optional, default=None)
        Seed for the random number generator

    Returns
    -------
    confidence interval : 1-d or 2-d array_like

    Example
    -------
    >>> rng = np.random.default_rng(12345)
    >>> y = 1 + rng.standard_normal(size=(10000, 4))
    >>> ci_calc(y, method='bootstrap', seed=rng)
    array([[0.98543573, 0.97812353, 0.99513357, 0.98174334],
           [1.02535726, 1.01852985, 1.03386319, 1.02152344]])

    >>> ci_calc(y, method='normal')
    array([[0.98530252, 0.97851995, 0.99611958, 0.98188726],
           [1.02378054, 1.01758363, 1.03539235, 1.02153012]])

    See Also
    --------
    https://github.com/BMClab/BMC/blob/master/notebooks/ConfidencePredictionIntervals.ipynb
    https://stats.stackexchange.com/questions/355781/
    """
    if method == 'bootstrap':
        rng = np.random.default_rng(seed=seed).integers
        if n is None:
            n = y.shape[axis]
        boots = [estimate(y.take(rng(y.shape[axis], size=n), axis=axis),
                          axis=axis) for i in range(n_boot)]
        ci = np.percentile(boots, [(100-level)/2, (100+level)/2], axis=0)
    elif method == 'bootstrap_sns':            
        import seaborn as sns
        ci = sns.utils.ci(sns.algorithms.bootstrap(y, n_boot=n_boot, axis=axis,
                                                   units=None, func=estimate,
                                                   seed=seed),
                          which=level, axis=axis)
    elif method == 'normal':
        from scipy import stats
        n = y.shape[axis]
        tppf = stats.t.ppf((1+level/100)/2, n-1)
        std = np.nanstd(y, axis=axis, ddof=1)
        ci = estimate(y, axis=axis) + tppf * np.array([-std, std])/np.sqrt(n)
    else:
        raise Exception('Unknown method for calculating ci: {}.'.format(method))

    return ci


def m_ci95(x=None, estimate=np.nanmedian, level=95, method='bootstrap'):
    """Return array with median and 95% confidence interval."""
    ci = ci_calc(x, estimate=estimate, level=level, method=method)
    return np.r_[estimate(x), ci[0], ci[1]]


def m_ci95df(y, estimate=np.nanmedian, level=level, method='bootstrap'):
    y = y.groupby(by='datetime', axis='index').apply(lambda x: m_ci95(x, estimate, level, method))
    y = pd.DataFrame(y.values.tolist(), index=y.index, columns=['m', 'c95inf', 'c95sup'])
    return y

## Load dataset

Let's load the dataset and create a dictionary with all the data and grouped by gender and age.  
We will also create two equivalents dictionaries with the number of athletes per period and the number of activities per athlete.

In [ ]:
dfs, dfs0, nathletes, nruns = dict(), dict(), dict(), dict()
for year in years:
    for f, frq in enumerate(freqs):
        fname = 'run_ww_{}_{}.parquet'.format(year, frq)
        df = pd.read_parquet(os.path.join(path2, fname))
        df['athlete'] = df['athlete'].astype('category')  # bug in parquet
        df = df.set_index('datetime', drop=True)
        #df.index = df.index.normalize()
        display(df.iloc[[0, -1]].style.set_caption('Year: {}, Frequency: {} (first and last rows)'.
                                                   format(year, freqs_s[f])))
        dfs[year, frq] = df
        dfs0[year, frq] = df[df[variables[0]] > 0]
        nathletes[year, frq] = dfs0[year, frq].groupby(level='datetime').size()
        x = dfs0[year, frq].groupby('athlete').size()
        nruns[year, frq] = x[x > 0]  # removes athletes with zero runs
        for gender in genders:
            for age in ages:
                x = dfs0[year, frq][(dfs0[year, frq]['gender'] == gender) &
                                     (dfs0[year, frq]['age_group'] == age)] 
                nathletes[year, frq, gender, age] = x.groupby(level='datetime').size()
                x = x.groupby('athlete').size()
                nruns[year, frq, gender, age] = x[x > 0]  # removes athletes with zero runs

### Basic information about the dataset

In [ ]:
dfs['2019', 'd'].info(memory_usage='deep')

In [ ]:
for year in years:
    print('Year: {}'.format(year))
    nday = dfs[year, 'd'].index.value_counts().size
    print(' Number of days:', nday)
    nathlete = nruns[year, 'd'].size
    print(' Number of athletes:', nathlete)
    nactivity = dfs[year, 'd'].shape[0]
    print(' Total number of activities:', nactivity)
    nactivity = nathletes[year, 'd'].sum()
    print(' Actual number of running activities:', nactivity)
print('Number of running activities in 2019 and 2020:',
      nathletes['2019', 'd'].sum() + nathletes['2020', 'd'].sum())

In [ ]:
nathletes['2019', 'd'].sum() + nathletes['2020', 'd'].sum()

In [ ]:
dfs0['2019', 'd'].shape[0] + dfs0['2020', 'd'].shape[0]

In [ ]:
nruns['2019', 'd'].sum() + nruns['2020', 'd'].sum()

## Load dataset with indexes of policy responses to the coronavirus pandemic

In [ ]:
c19idx = pd.read_csv(os.path.join(path2, 'policy_response_indexes.csv'), sep=',',
                 header=0, parse_dates=['date'], verbose=False)
c19idx = c19idx.set_index('date', drop=True)
c19idx

In [ ]:
def correct_last_days(y, freq):
    """
    Change the dates of the last days to avoid a last week with less than 7 days.
    """
    nlastdays = 7 + y.index[-1].dayofyear % 7
    if freq == '7d' and nlastdays > 7:
        ts = pd.Timestamp(y.index[-1].date() - pd.to_timedelta(nlastdays-7, unit='D'))
        y.index = y.index.where(y.index <= ts, ts)    
    grouper = pd.Grouper(axis=0, freq=freq)
    y = y.groupby(grouper).mean()
    if freq == '7d':
        # correct the divisor if the last week doesn't have 7 days
        y.loc[y.index[-1], :] = y.loc[y.index[-1], :] * (7 / nlastdays)
    return y

freq2 = '7d' if freq == 'w' else freq
c19idx = correct_last_days(c19idx, freq2)

## Running activity

In [ ]:
nathlete = nruns['2019', 'd'].size
nactivity = nathletes['2019', 'd'].sum() + nathletes['2020', 'd'].sum()

In [ ]:
grouper = pd.Grouper(axis=0, freq='y', sort=True)
for year in years:
    print(year)
    display(dfs[year, freq][['distance', 'duration']].groupby(grouper).describe().stack(level=0))    
    display(dfs[year, freq][['gender', 'age_group', 'distance', 'duration']].groupby(
        [grouper, 'gender', 'age_group']).describe().stack(level=0))

### Number of running activities per period

In [ ]:
print('Median and interquartile range of number of days the athletes ran each year:')
for year in years:
    print('In {}: {:.0f} ({:.0f}, {:.0f}) days'.
          format(year, *np.percentile(nruns[year, 'd'], [50, 25, 75]).round()))

The athletes ran on average about once every three days and half of them ran between once every week and once every two days in 2019 and 2020.

## Data distribution

### Difference between 2020 and 2019

In [ ]:
d_v = ['d_' + v for v in variables]
dfs['2020', freq][d_v] = ((dfs['2020', freq][variables].values -
                           dfs['2019', freq][variables].values) /
                          estimate(dfs['2019', freq][variables].values)) * 100

In [ ]:
display(dfs['2020', freq][d_v].describe().T)
display(dfs['2020', freq][d_v].groupby('datetime').describe())

### Histrogram

In [ ]:
if freq == 'd':
    nrows, ncols = 31, 12
    xmin, xmax = -20, 20
elif freq == 'w':
    nrows, ncols = 8, 7
    xmin, xmax = -50, 50
else:
    nrows, ncols = 4, 3  
    xmin, xmax = -200, 200

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(13, 6))
for v, var in enumerate(d_v):
    dfs['2020', freq][var].hist(bins=100, ax=ax[v])
    ax[v].set_title('{} (in {})'.format(var, units[v]))
fig.suptitle('Histogram of the difference (2020-2019)/2019 [%]')
plt.show()

In [ ]:
for v, var in enumerate(variables):
    fig, axs = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize=(13, 8))
    fig.suptitle('Histograms for {} difference at each {} (2020-2019)/2019 [%]'.format(var, freq))
    axs[0, 0].set_xlim(xmin - 6*v*xmin, xmax + 6*v*xmin)
    axs = axs.flatten()
    x = dfs['2020', freq][['athlete', 'd_'+var]].pivot(columns='athlete')['d_'+var].T
    axs = x.hist(bins=100, figsize=(13, 9), ax=axs[:x.shape[1]])
    for ax in axs:
        ax.set_title('')
    plt.tight_layout(h_pad=.05, w_pad=.05)
    plt.show()

## Volume of training: average and 95% confidence interval

### Calculus of central tendency and confidence interval for each year

In [ ]:
dfsci, dfs0ci = dict(), dict()

for var in variables:
    for year in years:
        dfsci[year, freq, var] = m_ci95df(y=dfs[year, freq][var],
                                          estimate=estimate, level=level, method=method)
        dfs0ci[year, freq, var] = m_ci95df(y=dfs0[year, freq][var],
                                           estimate=estimate, level=level, method=method)
        for gender in genders:
            for age in ages:
                x = dfs[year, freq][(dfs[year, freq]['gender'] == gender) &
                                    (dfs[year, freq]['age_group'] == age)][var]
                dfsci[year, freq, gender, age, var] = m_ci95df(y=x, estimate=estimate,
                                                               level=level, method=method)
                x = dfs0[year, freq][(dfs0[year, freq]['gender'] == gender) &
                                     (dfs0[year, freq]['age_group'] == age)][var]
                dfs0ci[year, freq, gender, age, var] = m_ci95df(y=x, estimate=estimate,
                                                                level=level, method=method)             

### Calculate differences between years

In [ ]:
dfscid = dict()
nathletesd = dict()
# store old index
dfs['2019', freq]['index'] = dfs['2019', freq].index
dfs['2020', freq]['index'] = dfs['2020', freq].index
days = np.r_[0, np.cumsum(dfs['2019', 'm'].index.unique().daysinmonth.values)[:]]
# change the index
idx = dfs['2019', freq].index.dayofyear
dfs['2019', freq].index = idx
dfs['2020', freq].index = idx

for var in variables:
    dfscid[freq, var] = m_ci95df(100 * ((dfs['2020', freq][var] - dfs['2019', freq][var]) /
                                        estimate(dfs['2019', freq][var])),
                                 estimate=estimate, level=level, method=method)

    nathletesd[freq, var] = pd.Series(((nathletes['2020', freq].values - nathletes['2019', freq].values) /
                                     estimate(nathletes['2019', freq])) * 100,
                                     index=dfscid[freq, var].index)

    for gender in genders:
        for age in ages:
            idx = (dfs['2020', freq]['gender'] == gender) & (dfs['2020', freq]['age_group'] == age)
            x = 100 * ((dfs['2020', freq][idx][var] - dfs['2019', freq][idx][var]) /
                       estimate(dfs['2019', freq][idx][var]))
            dfscid[freq, gender, age, var] = m_ci95df(x, estimate=estimate, level=level, method=method)   

            nathletesd[freq, gender, age, var] = pd.Series(((nathletes['2020', freq, gender, age].values -  
                                                      nathletes['2019', freq, gender, age].values) /
                                                      estimate(nathletes['2019', freq, gender, age]))
                                                      * 100, index=dfscid[freq, var].index)

In [ ]:
for var in variables:
    print(var)
    display(dfscid[freq, var]['m'].describe().to_frame(name='All athletes').T)
    display(nathletesd[freq, var].describe().to_frame(name='N All athletes').T)
    for gender in genders:
        for age in ages:
            display(dfscid[freq, gender, age, var]['m'].describe().to_frame(name=gender + age).T)
            display(nathletesd[freq, gender, age, var].describe().to_frame(name='N All athletes').T)

In [ ]:
for var in variables:
    print(var)
    display(nathletesd[freq, var].describe().to_frame().T)    
    display(nathletesd[freq, var].groupby('datetime').describe())

#### Statistical inference

In [ ]:
# demonstration
spm = spmtest(dfs['2020', freq][['athlete', variables[0]]].pivot(columns='athlete')[variables[0]].T,
              dfs['2019', freq][['athlete', variables[0]]].pivot(columns='athlete')[variables[0]].T)
spmi = spm.inference(alpha, two_tailed=two_tailed)
print(spmi)
# print(snpmi.clusters)

ax = plt.axes((0.55, 0.15, 0.7, 0.8))
spmi.plot()
spmi.plot_threshold_label(fontsize=8)
spmi.plot_p_values(size=10)
ax.set_xlabel('Time (weeks)')
plt.show()

In [ ]:
spmi = dict()
for var in variables:
    spm = spmtest(dfs['2020', freq][['athlete', var]].pivot(columns='athlete')[var].T,
                  dfs['2019', freq][['athlete', var]].pivot(columns='athlete')[var].T)
    spmi[freq, var] = spm.inference(alpha=alpha, two_tailed=two_tailed)
    for gender in genders:
        for age in ages:
            idx = (dfs['2020', freq]['gender']==gender) & (dfs['2020', freq]['age_group']==age)
            spm = spmtest(dfs['2020', freq][idx][['athlete', var]].pivot(columns='athlete')[var].T,
                          dfs['2019', freq][idx][['athlete', var]].pivot(columns='athlete')[var].T)
            spmi[freq, gender, age, var] = spm.inference(alpha=alpha, two_tailed=two_tailed)   

### Plots

In [ ]:
def repeat_last(df, freq):
    """Repeat last point in data for plotting.
    """    
    x = df.iloc[-1]
    if freq == 'w':
        x.name = x.name + 7
        df = df.append(x)
        t = df.index
    elif freq == 'm':
        x.name = x.name + 31
        df = df.append(x)
        t = df.index - 31
    return t, df


def plot(ax, axb, freq, df, n, c1=c1, c2=c2):
    """General plot.
    """
    t, df = repeat_last(df, freq)
    n = np.r_[n, n[-1]]
    ax.fill_between(t, df['c95inf'], df['c95sup'],
                    color=c1, alpha=.7, clip_on=False)  # , step='post'
    sns.lineplot(x=t, y=df['m'], color=c1, ci=None, alpha=1, 
                 legend=False, lw=3, ax=ax, clip_on=False) # , drawstyle='steps-post'
    axb.plot(t, n, color=c2, lw=3, alpha=1, clip_on=False)  # , drawstyle='steps-post'


weight = 'normal'
x = days

for v, var in enumerate(variables):
    fig = plt.figure(constrained_layout=True, figsize=(12, 8))
    gs = fig.add_gridspec(3, 3)
    ax0 = fig.add_subplot(gs[0, :])
    ax1 = fig.add_subplot(gs[1, 0])
    ax2 = fig.add_subplot(gs[1, 1])
    ax3 = fig.add_subplot(gs[1, 2])
    ax4 = fig.add_subplot(gs[2, 0])
    ax5 = fig.add_subplot(gs[2, 1])
    ax6 = fig.add_subplot(gs[2, 2])
    ax0b = ax0.twinx()  # ax0.get_shared_x_axes().get_siblings(ax0)[7]

    ax0.axvline(x[3], c=c3, alpha=.8, ls=':', lw=1)
    ax0.axvline(x[6], c=c3, alpha=.8, ls=':', lw=1)
    ax0.axvline(x[9], c=c3, alpha=.8, ls=':', lw=1)
    ax0.axhline(0, c=c3, alpha=.8, lw=2)

    if 1:
        plot(ax0, ax0b, freq, dfscid[freq, var], nathletesd[freq, var].values, c1, c2)
    else:
        width = 4
        t = dfscid[freq, var].index
        #yerr = np.diff(ci, axis=0).flatten() * norm[days].values/2
        ax0.bar(t - width/2, dfscid[freq, var]['m'], width=width, linewidth=0, color=c1)
        ax0b.bar(t + width/2, r_a, width=width, linewidth=0, color=c2)

    # plot coronavirus policy response index
    ax0c = ax0.twinx()
    c19idx.index = dfscid[freq, var].index
    t, c19idx2 = repeat_last(c19idx, freq)
    ax0c.plot(t, c19idx2['si_w'], color=c4, alpha=.8, 
              lw=3, ls='-', clip_on=False)
    ax0c.set_ylim([100, -100])
    ax0c.set_yticks([0, 50, 100])    

    # spm1d plot
    # lines stretches from lineoffset - linelength/2 to lineoffset + linelength/2
    if freq == 'd':
        factor = 1
    elif freq == 'w':
        factor = 7
    elif freq == 'm':
        factor = 31
    s2020 = np.where(spmi[freq, var].z > spmi[freq, var].zstar)[0] 
    s2020 = (t[s2020] + factor/2, ) 
    s2019 = np.where(spmi[freq, var].z < -spmi[freq, var].zstar)[0]
    s2019 = (t[s2019] + factor/2, ) 
    ax0.eventplot(-np.ones_like(s2020)*52, colors=c1, lineoffsets=s2020,
                  linelengths=[factor], orientation='vertical',
                  linewidth=6, alpha=0.4, clip_on=False)
    ax0.eventplot(-np.ones_like(s2019)*58, colors=c1, lineoffsets=s2019,
                  linelengths=[factor], orientation='vertical',
                  linewidth=6, alpha=0.4, clip_on=False)

    axes = np.array([[ax1, ax2, ax3], [ax4, ax5, ax6]])
    axesb = np.empty_like(axes)
    letter = np.array([['b', 'c', 'd'], ['e', 'f', 'g']])
    for g, gender in enumerate(genders):
        for a, age in enumerate(ages):
            axesb[g, a] = axes[g, a].twinx()
            axes[g, a].axvline(x[3], c=c3, alpha=.8, ls=':', lw=1)
            axes[g, a].axvline(x[6], c=c3, alpha=.8, ls=':', lw=1)
            axes[g, a].axvline(x[9], c=c3, alpha=.8, ls=':', lw=1)
            axes[g, a].axhline(0, c=c3, alpha=.8, lw=2)
            plot(axes[g, a], axesb[g, a], freq, dfscid[freq, gender, age, var],
                  nathletesd[freq, gender, age, var].values, c1, c2)
            # spm1d plot
            # lines stretches from lineoffset - linelength/2 to lineoffset + linelength/2
            s2020 = np.where(spmi[freq, gender, age, var].z >
                             spmi[freq, gender, age, var].zstar)[0] 
            s2020 = (t[s2020] + factor/2, ) 
            s2019 = np.where(spmi[freq, gender, age, var].z <
                             -spmi[freq, gender, age, var].zstar)[0]
            s2019 = (t[s2019] + factor/2, )    
            axes[g, a].eventplot(-np.ones_like(s2020)*52, colors=c1, lineoffsets=s2020,
                                 linelengths=[factor], orientation='vertical',
                                 linewidth=6, alpha=0.4, clip_on=False)
            axes[g, a].eventplot(-np.ones_like(s2019)*58, colors=c1, lineoffsets=s2019,
                                 linelengths=[factor], orientation='vertical',
                                 linewidth=6, alpha=0.4, clip_on=False)
            # aesthetics
            axes[g, a].text(-.045, 1.025, letter[g, a], ha='right', weight=weight,
                            fontsize=14, c='k', transform=axes[g, a].transAxes)
            axes[g, a].text(0.02, .9, '{:.0f} {}'.format(np.round(estimate(
                            dfsci['2020', freq, gender, age, var]['m']), 0), units[v]),
                            ha='left', weight=weight, fontsize=14, c=c1,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(.98, .9, '{:,}'.format(nruns['2020', freq, gender, age].size),
                            ha='right', weight=weight, fontsize=14, c=c2,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(0.02, .78, '{:.0f} {}'.format(np.round(estimate(
                            dfsci['2019', freq, gender,age, var]['m']), 0), units[v]),
                            ha='left', weight=weight, fontsize=14, c=c1,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(.98, .78, '{:,}'.format(nruns['2019', freq, gender, age].size),
                            ha='right', weight=weight, fontsize=14, c=c2,
                            transform=axes[g, a].transAxes)
            gender2 = 'Women' if gender == 'F' else 'Men'
            age2 = age + ' yr.'
            axes[g, a].set_title('{} {}'.format(gender2, age2), fontsize=14,
                                 c=c3, ha='center', y=1)
            axes[g, a].set_xticks(x)
            if g == 0:
                axes[g, a].set_xticklabels([])
            if g == 1:
                labels = ['  ' + m[0] for m in months]
                labels.append('')
                axes[g, a].set_xticklabels(labels, rotation=0, fontsize=14, ha='left')
            if a == 0:
                axes[g, a].set_ylabel('')
                #axesb[g, a].axes.get_yaxis().set_visible(False)
            if a > 0:
                axes[g, a].set_yticklabels([])
                axes[g, a].set_ylabel('')
                #axes[g, a].spines['left'].set_visible(False)
            if a == 1:
                #axes[g, a].axes.get_yaxis().set_visible(False)
                #axesb[g, a].axes.get_yaxis().set_visible(False)
                pass
            if a == 2:
                #axes[g, a].axes.get_yaxis().set_visible(False)
                pass
            if a < 2:
                axesb[g, a].set_yticklabels([])
                axesb[g, a].set_ylabel('')
                #axesb[g, a].spines['right'].set_visible(False)
            if g == 1 & a == 1:
                axes[g, a].set_xlabel('Month of the year', c=c3, fontsize=16)
                axes[g, a].xaxis.set_label_coords(.5, -.3)
            else:
                axes[g, a].set_xlabel('')

            axes[g, a].tick_params(axis='x', colors=c3)
            axes[g, a].spines['right'].set_visible(False)
            axes[g, a].spines['top'].set_visible(False)
            axesb[g, a].spines['bottom'].set_visible(False)
            axesb[g, a].spines['left'].set_visible(False)
            axesb[g, a].spines['top'].set_visible(False)
            axes[g, a].spines['bottom'].set_color(c3)
            axes[g, a].spines['left'].set_color(c1)
            axes[g, a].tick_params(axis='y', colors=c1)
            axesb[g, a].spines['right'].set_color(c2)
            axesb[g, a].tick_params(axis='y', colors=c2)
            axes[g, a].spines['bottom'].set_position(('axes', -0.15))
            axes[g, a].spines['left'].set_position(('axes', -0.04))
            axesb[g, a].spines['right'].set_position(('axes', 1.04))
            axes[g, a].xaxis.set_ticks_position('bottom')
            axes[g, a].yaxis.set_ticks_position('left')
            axesb[g, a].xaxis.set_ticks_position('bottom')
            axesb[g, a].yaxis.set_ticks_position('right')   
            axes[g, a].tick_params(axis='y', which='major', labelsize=14)
            axesb[g, a].tick_params(axis='y', which='major', labelsize=14)

    ax0.set_ylabel('')
    ax0.set_xlabel('')
    ax0.xaxis.set_ticks_position('bottom')
    ax0.spines['bottom'].set_color(c3)
    ax0.spines['top'].set_visible(False)
    ax0.spines['right'].set_visible(False)
    ax0.spines['left'].set_color(c1)
    ax0.tick_params(axis='x', colors=c3)
    ax0.tick_params(axis='y', colors=c1)
    ax0.spines['left'].set_position(('axes', -0.012))
    ax0.spines['bottom'].set_position(('axes', -0.15))

    ax0b.spines['bottom'].set_visible(False)
    ax0b.spines['top'].set_visible(False)
    ax0b.spines['left'].set_visible(False)
    ax0b.spines['right'].set_color(c2)
    ax0b.spines['right'].set_position(('axes', 1.012))
    ax0b.tick_params(axis='y', colors=c2)

    ax0c.spines['bottom'].set_visible(False)
    ax0c.spines['top'].set_visible(False)
    ax0c.spines['left'].set_visible(False)
    ax0c.spines['right'].set_color(c4)
    ax0c.spines['right'].set_position(('axes', 1.095))
    ax0c.tick_params(axis='y', colors=c4)
    ax0c.tick_params(axis='both', which='major', labelsize=14)
    ax0c.set_ylabel('2020 stringency index', rotation=-90, color=c4, fontsize=16)
    ax0c.yaxis.set_label_coords(1.16, .45)
    ax0c.grid(False, axis='both')

    ax0.set_title('All athletes', fontsize=16, c=c3, ha='center')
    ax0.text(.065, 1.04, 'Mean {}'.format(var), ha='center', weight=weight,
             fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(.91, 1.04, 'Number of athletes', ha='center', weight=weight,
             fontsize=14, c=c2, transform=ax0.transAxes)
    ax0.text(-.0155, 1.025, 'a', ha='right', weight=weight,
             fontsize=14, c='k', transform=ax0.transAxes)
    ax0.text(0.014, .9, '2020:', ha='left', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(0.07, .9, '{:.0f} {}'.format(
        np.round(estimate(dfsci['2020', freq, var]['m']), 0), units[v]),
             ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.91, .9, '2020:', ha='right', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(.975, .9,  '{:,}'.format(nruns['2020', 'd'].size), ha='right', weight=weight,
             fontsize=14, c=c2, transform=ax0.transAxes)
    ax0.text(0.014, .78, '2019:', ha='left', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(0.07, .78, '{:.0f} {}'.format(
             np.round(estimate(dfsci['2019', freq, var]['m']), 0), units[v]),
             ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.91, .78, '2019:', ha='right', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(.975, .78, '{:,}'.format(nruns['2019', 'd'].size), ha='right', weight=weight,
             fontsize=14, c=c2, transform=ax0.transAxes)

    if freq == 'd':
        sfrq = 'Daily'
    elif freq == 'w':
        sfrq = 'Weekly'
    elif freq == 'm':
        sfrq = 'Monthly'
    text = '{} running volume difference between 2020 and 2019 [%]'.format(sfrq)
    #text = sfrq + ' running volume $\mathregular{(V_{2020}-V_{2019})\:/\:\overline{V_{2019}}}$  [%]'
    ax1.set_ylabel(text, color=c1, fontsize=16)
    ax1.yaxis.set_label_coords(-.18, .5)
    text = '{} number of athletes difference between 2020 and 2019 [%]'.format(sfrq)
    #text = sfrq + ' number of athletes $\mathregular{(N_{2020}-N_{2019})\:/\:\overline{N_{2019}}}$  [%]'
    axesb[0, 2].set_ylabel(text, rotation=-90, color=c2, fontsize=16)
    axesb[0, 2].yaxis.set_label_coords(1.27, .5)
    ax0.tick_params(axis='both', which='major', labelsize=14)
    ax0b.tick_params(axis='both', which='major', labelsize=14)
    ax0.set_xticks(x)
    labels = [m[0:3] for m in months]
    labels.append('')
    ax0.set_xticklabels(labels, rotation=0, fontsize=14, ha='left')

    axes2 = np.r_[ax0, ax1, ax2, ax3, ax4, ax5, ax6]
    axesb2 = np.r_[ax0b, axesb.flatten()]
    for ax, axb in zip(axes2, axesb2):
        ax.grid(False, axis='both')
        axb.grid(False, axis='both')
        ax.set_xlim([0, 365])
        ax.set_ylim([-50, 50])
        ax.set_yticks([-40, -20, 0, 20, 40])
        axb.set_ylim([-25, 25])
        axb.set_yticks([-20, -10, 0, 10, 20])
    fig.savefig(os.path.join(path2, 'figure1_{}.svg'.format(var)), dpi=600)
    fig.savefig(os.path.join(path2, 'figure1_{}.png'.format(var)), dpi=600)
    plt.show()

#### Infographic

In [ ]:
def repeat_last(df, freq):
    """Repeat last point in data for plotting.
    """    
    x = df.iloc[-1]
    if freq == 'w':
        x.name = x.name + 7
        df = df.append(x)
        t = df.index
    elif freq == 'm':
        x.name = x.name + 31
        df = df.append(x)
        t = df.index - 31
    return t, df


def plot(ax, axb, freq, df, n, c1=c1, c2=c2):
    """General plot.
    """
    t, df = repeat_last(df, freq)
    n = np.r_[n, n[-1]]
    ax.fill_between(t, df['c95inf'], df['c95sup'],
                    color=c1, alpha=.7, clip_on=False)  # , step='post'
    sns.lineplot(x=t, y=df['m'], color=c1, ci=None, alpha=1, 
                 legend=False, lw=4, ax=ax, clip_on=False) # , drawstyle='steps-post'
    axb.plot(t, n, color=c2, lw=4, alpha=1, clip_on=False)  # , drawstyle='steps-post'


weight = 'normal'
x = days

for v, var in enumerate(variables):
    fig = plt.figure(constrained_layout=False, figsize=(13, 13))
    gs = fig.add_gridspec(3, 3)
    ax0 = fig.add_subplot(gs[0, :])
    ax1 = fig.add_subplot(gs[1, 0])
    ax2 = fig.add_subplot(gs[1, 1])
    ax3 = fig.add_subplot(gs[1, 2])
    ax4 = fig.add_subplot(gs[2, 0])
    ax5 = fig.add_subplot(gs[2, 1])
    ax6 = fig.add_subplot(gs[2, 2])
    ax0b = ax0.twinx()  # ax0.get_shared_x_axes().get_siblings(ax0)[7]
    ax0c = ax0.twinx()

    #ax0.axvline(x[3], c=c3, alpha=.8, ls=':', lw=1)
    #ax0.axvline(x[6], c=c3, alpha=.8, ls=':', lw=1)
    #ax0.axvline(x[9], c=c3, alpha=.8, ls=':', lw=1)
    ax0.axvspan(x[0], x[3], fc=c5, alpha=0.15)
    ax0.axvspan(x[3], x[6], fc=c5, alpha=0.05)
    ax0.axvspan(x[6], x[9], fc=c5, alpha=0.15)
    ax0.axvspan(x[9], x[12], fc=c5, alpha=0.05)
    ax0.axhline(0, c=c3, alpha=.8, lw=2)

    if 1:
        plot(ax0, ax0b, freq, dfscid[freq, var], nathletesd[freq, var].values, c1, c2)
    else:
        width = 4
        t = dfscid[freq, var].index
        #yerr = np.diff(ci, axis=0).flatten() * norm[days].values/2
        ax0.bar(t - width/2, dfscid[freq, var]['m'], width=width, linewidth=0, color=c1)
        ax0b.bar(t + width/2, r_a, width=width, linewidth=0, color=c2)

    # plot coronavirus policy response index
    c19idx.index = dfscid[freq, var].index
    t, c19idx2 = repeat_last(c19idx, freq)
    ax0c.plot(t, c19idx2['si_w'], color=c4, alpha=.8, 
              lw=4, ls='-', clip_on=False)
    ax0c.set_ylim([100, -100])
    ax0c.set_yticks([0, 50, 100])    

    # spm1d plot
    # lines stretches from lineoffset - linelength/2 to lineoffset + linelength/2
    if freq == 'd':
        factor = 1
    elif freq == 'w':
        factor = 7
    elif freq == 'm':
        factor = 31
    s2020 = np.where(spmi[freq, var].z > spmi[freq, var].zstar)[0] 
    s2020 = (t[s2020] + factor/2, ) 
    s2019 = np.where(spmi[freq, var].z < -spmi[freq, var].zstar)[0]
    s2019 = (t[s2019] + factor/2, ) 
    ax0.eventplot(-np.ones_like(s2020)*52, colors=c1, lineoffsets=s2020,
                  linelengths=[factor], orientation='vertical',
                  linewidth=6, alpha=0.4, clip_on=False)
    ax0.eventplot(-np.ones_like(s2019)*58, colors=c1, lineoffsets=s2019,
                  linelengths=[factor], orientation='vertical',
                  linewidth=6, alpha=0.4, clip_on=False)

    axes = np.array([[ax1, ax2, ax3], [ax4, ax5, ax6]])
    axesb = np.empty_like(axes)
    letter = np.array([['B', 'C', 'D'], ['E', 'F', 'G']])
    for g, gender in enumerate(genders):
        for a, age in enumerate(ages):
            axesb[g, a] = axes[g, a].twinx()
            #axes[g, a].axvline(x[3], c=c3, alpha=.8, ls=':', lw=1)
            #axes[g, a].axvline(x[6], c=c3, alpha=.8, ls=':', lw=1)
            #axes[g, a].axvline(x[9], c=c3, alpha=.8, ls=':', lw=1)
            axes[g, a].axvspan(x[0], x[3], fc=c5, alpha=0.15)
            axes[g, a].axvspan(x[3], x[6], fc=c5, alpha=0.05)
            axes[g, a].axvspan(x[6], x[9], fc=c5, alpha=0.15)
            axes[g, a].axvspan(x[9], x[12], fc=c5, alpha=0.05)
            axes[g, a].axhline(0, c=c3, alpha=.8, lw=2)
            plot(axes[g, a], axesb[g, a], freq, dfscid[freq, gender, age, var],
                 nathletesd[freq, gender, age, var].values, c1, c2)
            # spm1d plot
            # lines stretches from lineoffset - linelength/2 to lineoffset + linelength/2
            s2020 = np.where(spmi[freq, gender, age, var].z >
                             spmi[freq, gender, age, var].zstar)[0] 
            s2020 = (t[s2020] + factor/2, ) 
            s2019 = np.where(spmi[freq, gender, age, var].z <
                             -spmi[freq, gender, age, var].zstar)[0]
            s2019 = (t[s2019] + factor/2, )    
            axes[g, a].eventplot(-np.ones_like(s2020)*52, colors=c1, lineoffsets=s2020,
                                 linelengths=[factor], orientation='vertical',
                                 linewidth=6, alpha=0.4, clip_on=False)
            axes[g, a].eventplot(-np.ones_like(s2019)*58, colors=c1, lineoffsets=s2019,
                                 linelengths=[factor], orientation='vertical',
                                 linewidth=6, alpha=0.4, clip_on=False)
            # aesthetics
            #axes[g, a].text(-.04, 1.02, letter[g, a], ha='right', weight=weight,
            #                fontsize=16, c='k', transform=axes[g, a].transAxes)
            axes[g, a].text(0.02, .9, '{:.0f} {}'.format(np.round(estimate(
                            dfsci['2020', freq, gender, age, var]['m']), 0), units[v]),
                            ha='left', weight=weight, fontsize=14, c=c1,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(.98, .9, '{:,}'.format(nruns['2020', freq, gender, age].size),
                            ha='right', weight=weight, fontsize=14, c=c2,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(0.02, .78, '{:.0f} {}'.format(np.round(estimate(
                            dfsci['2019', freq, gender, age, var]['m']), 0), units[v]),
                            ha='left', weight=weight, fontsize=14, c=c1,
                            transform=axes[g, a].transAxes)
            axes[g, a].text(.98, .78, '{:,}'.format(nruns['2019', freq, gender, age].size),
                            ha='right',  weight=weight, fontsize=14, c=c2,
                            transform=axes[g, a].transAxes)
            gender2 = 'Women' if gender == 'F' else 'Men'
            age2 = age + ' yr.'
            axes[g, a].set_title('{} {}'.format(gender2, age2), fontsize=14,
                                 c=c3, ha='center', y=1)
            axes[g, a].set_xticks(x)
            if g == 0:
                axes[g, a].axes.get_xaxis().set_visible(False)
                axes[g, a].spines['bottom'].set_visible(False)
                axes[g, a].set_xticklabels([])
            if g == 1:
                labels = ['  ' + m[0] for m in months]
                labels.append('')
                axes[g, a].set_xticklabels(labels, rotation=0, fontsize=14, ha='left')
            if a == 0:
                axes[g, a].set_ylabel('')
                #axesb[g, a].axes.get_yaxis().set_visible(False)
            if a > 0:
                axes[g, a].set_yticklabels([])
                axes[g, a].set_ylabel('')
                #axes[g, a].spines['left'].set_visible(False)
            if a == 1:
                #axes[g, a].axes.get_yaxis().set_visible(False)
                #axesb[g, a].axes.get_yaxis().set_visible(False)
                pass
            if a == 2:
                #axes[g, a].axes.get_yaxis().set_visible(False)
                pass
            if a < 2:
                axesb[g, a].set_yticklabels([])
                axesb[g, a].set_ylabel('')
                #axesb[g, a].spines['right'].set_visible(False)
            if g == 1 & a == 1:
                axes[g, a].set_xlabel('Month of the year', c=c3, fontsize=18)
                axes[g, a].xaxis.set_label_coords(.5, -.28)
            else:
                axes[g, a].set_xlabel('')

            axes[g, a].tick_params(axis='x', colors=c3)
            axes[g, a].spines['right'].set_visible(False)
            axes[g, a].spines['top'].set_visible(False)
            axesb[g, a].spines['bottom'].set_visible(False)
            axesb[g, a].spines['left'].set_visible(False)
            axesb[g, a].spines['top'].set_visible(False)
            axes[g, a].spines['bottom'].set_color(c3)
            axes[g, a].spines['left'].set_color(c1)
            axes[g, a].tick_params(axis='y', colors=c1)
            axesb[g, a].spines['right'].set_color(c2)
            axesb[g, a].tick_params(axis='y', colors=c2)
            axes[g, a].spines['bottom'].set_position(('axes', -0.15))
            axes[g, a].spines['left'].set_position(('axes', -0.04))
            axesb[g, a].spines['right'].set_position(('axes', 1.04))
            axes[g, a].xaxis.set_ticks_position('bottom')
            axes[g, a].yaxis.set_ticks_position('left')
            axesb[g, a].xaxis.set_ticks_position('bottom')
            axesb[g, a].yaxis.set_ticks_position('right')   
            axes[g, a].tick_params(axis='y', which='major', labelsize=14)
            axesb[g, a].tick_params(axis='y', which='major', labelsize=14)

    ax0.set_ylabel('')
    ax0.set_xlabel('')
    ax0.xaxis.set_ticks_position('bottom')
    ax0.spines['bottom'].set_color(c3)
    ax0.spines['top'].set_visible(False)
    ax0.spines['right'].set_visible(False)
    ax0.spines['left'].set_color(c1)
    ax0.tick_params(axis='x', colors=c3)
    ax0.tick_params(axis='y', colors=c1)
    ax0.spines['left'].set_position(('axes', -0.012))
    ax0.spines['bottom'].set_position(('axes', -0.15))

    ax0b.spines['bottom'].set_visible(False)
    ax0b.spines['top'].set_visible(False)
    ax0b.spines['left'].set_visible(False)
    ax0b.spines['right'].set_color(c2)
    ax0b.spines['right'].set_position(('axes', 1.012))
    ax0b.tick_params(axis='y', colors=c2)

    ax0c.spines['bottom'].set_visible(False)
    ax0c.spines['top'].set_visible(False)
    ax0c.spines['left'].set_visible(False)
    ax0c.spines['right'].set_color(c4)
    ax0c.spines['right'].set_position(('axes', 1.12))
    ax0c.tick_params(axis='y', colors=c4)
    ax0c.tick_params(axis='both', which='major', labelsize=14)
    ax0c.set_ylabel('2020 stringency index$^{[2]}$', rotation=-90, color=c4, fontsize=17)
    ax0c.yaxis.set_label_coords(1.2, .44)
    ax0c.grid(False, axis='both')

    ax0.set_title('All athletes', fontsize=16, c=c3, ha='center')
    #ax0.text(-.015, 1.02, 'A', ha='right', weight=weight,
    #         fontsize=16, c='k', transform=ax0.transAxes)
    ax0.text(.08, 1.04, 'Mean {}'.format(var), ha='center', weight=weight,
             fontsize=16, c=c1, transform=ax0.transAxes)
    ax0.text(.91, 1.04, 'Number of athletes', ha='center', weight=weight,
             fontsize=16, c=c2, transform=ax0.transAxes)
    ax0.text(0.01, .9, '2020:', ha='left', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    #ax0.text(0.055, .91, '$\mathregular{\overline{V}}$', 
    #         ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.07, .9, '{:.0f} {}'.format(
             np.round(estimate(dfsci['2020', freq, var]['m']), 0), units[v]),
             ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.915, .9, '2020:', ha='right', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(.985, .9,  '{:,}'.format(nruns['2020', 'd'].size), ha='right', weight=weight,
             fontsize=14, c=c2, transform=ax0.transAxes)
    ax0.text(0.01, .78, '2019:', ha='left', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    #ax0.text(0.055, .8, '$\mathregular{\overline{V}}$', 
    #         ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.07, .78, '{:.0f} {}'.format(
             np.round(estimate(dfsci['2019', freq, var]['m']), 0), units[v]),
             ha='left', weight=weight, fontsize=14, c=c1, transform=ax0.transAxes)
    ax0.text(0.915, .78, '2019:', ha='right', weight=weight,
             fontsize=14, c=c3, transform=ax0.transAxes)
    ax0.text(.985, .78, '{:,}'.format(nruns['2019', 'd'].size), ha='right', weight=weight,
             fontsize=14, c=c2, transform=ax0.transAxes)

    if freq == 'd':
        sfrq = 'Daily'
    elif freq == 'w':
        sfrq = 'Weekly'
    elif freq == 'm':
        sfrq = 'Monthly'
    text = '{} running volume difference between 2020 and 2019 [%]'.format(sfrq)
    #text = sfrq + ' running volume $\mathregular{(V_{2020}-V_{2019})\:/\:\overline{V_{2019}}}$  [%]'
    ax1.set_ylabel(text, color=c1, fontsize=19)
    ax1.yaxis.set_label_coords(-.21, .5)
    text = '{} number of athletes difference between 2020 and 2019 [%]'.format(sfrq)
    #text = sfrq + ' number of athletes $\mathregular{(N_{2020}-N_{2019})\:/\:\overline{N_{2019}}}$  [%]'
    axesb[0, 2].set_ylabel(text, rotation=-90, color=c2, fontsize=19)
    axesb[0, 2].yaxis.set_label_coords(1.31, .5)
    ax0.tick_params(axis='both', which='major', labelsize=14)
    ax0b.tick_params(axis='both', which='major', labelsize=14)
    ax0.set_xticks(x)
    labels = [m[0:3] for m in months]
    labels.append('')
    ax0.set_xticklabels(labels, rotation=0, fontsize=14, ha='left')

    axes2 = np.r_[ax0, ax1, ax2, ax3, ax4, ax5, ax6]
    axesb2 = np.r_[ax0b, axesb.flatten()]
    for ax, axb in zip(axes2, axesb2):
        ax.grid(False, axis='both')
        axb.grid(False, axis='both')
        ax.set_xlim([0, 365])
        ax.set_ylim([-50, 50])
        ax.set_yticks([-40, -20, 0, 20, 40])
        axb.set_ylim([-25, 25])
        axb.set_yticks([-20, -10, 0, 10, 20])
        #ax.patch.set_facecolor([.9, .9, .9])

    #title = 'Worldwide weekly training of long distance running$^{[1]}$ in 2019 and 2020\n' + \
    #        '— Associated effects of the COVID-19 pandemic —'
    #fig.suptitle(title, color='w', fontsize=25, weight='semibold', x=.51, y=.985)

    text = ['Worldwide weekly training of long-distance running$^{[1]}$ in 2019 and 2020']
    fig.text(0.5, .96, text[0], fontsize=26, color='w', weight='semibold',
             horizontalalignment='center')
    text = ['A within-subject comparison for the investigation of the COVID-19 pandemic associated effects']
    fig.text(0.5, .92, text[0], fontsize=20, color='w', weight='semibold',
             style='italic', horizontalalignment='center')


    #ax = fig.add_axes([0, 0.92, 1, .1])
    #ax.xaxis.set_visible(False)
    #ax.yaxis.set_visible(False)
    #ax.set_zorder(0)
    #ax.patch.set_alpha(0.5)
    #ax.patch.set_color([.2, .2, .2])

    fig.patches.extend([plt.Rectangle((0, .9), 1, .1, color=c5, fill=True, alpha=.9,
                                      zorder=0, transform=fig.transFigure, figure=fig)])
    fig.patches.extend([plt.Rectangle((0, 0), 1, .15, color=c5, fill=True, alpha=.9,
                                      zorder=0, transform=fig.transFigure, figure=fig)])
    fig.patches.extend([plt.Rectangle((0.014, 0.03), .015, .008, color=c1,
                                      fill=True, alpha=.4,
                                      zorder=10, transform=fig.transFigure, figure=fig)])

    fig.text(0.01, .13, 'Notes:', fontsize=14, color='w', weight='normal')
    text = [' [1]: Mean ($\pm$ 99%CI) based on data of 9,259,153 running activities of 36,412 athletes.' +
            ' All athletes ran at least one of the World Marathon Majors.']
    fig.text(0.01, .11, text[0], fontsize=14, color='w', alpha=1, weight='normal')        
    text = ['       Plots for all athletes and grouped by sex and age.' +
            ' Data extracted by web scraping the athletes\' public pages in' +
            ' https://www.strava.com.']
    fig.text(0.01, .09, text[0], fontsize=14, color='w', alpha=1, weight='normal')
    text = [' [2]: 2020 stringency index is an average of government responses to' +
            ' COVID-19 pandemic weighted by the number of athletes in each country.\n' +
            '       Data from https://ourworldindata.org/covid-government-stringency-index.']
    fig.text(0.01, .05, text[0], fontsize=14, color='w', alpha=1, weight='normal')
    text = ['     : The bars at the bottom of the plots indicate statistical ' +
            'difference in running volume between the marked weeks of 2020 and 2019 (α=0.01).']
    fig.text(0.01, .03, text[0], fontsize=14, color='w', alpha=1, weight='normal')
    text = ['By: L. Afonseca, M. Duarte, and R. Watanabe.' +
            ' More information in https://github.com/BMClab/covid19.']
    fig.text(0.01, .01, text[0], fontsize=14, color='w', weight='normal')

    #ax0.text(.01, -.1, '[3]', ha='right', weight='normal', fontsize=16, c=c1, transform=ax0.transAxes)

    axes2[0].set_position([0.13, 0.70, 0.71, 0.17])
    axes2[1].set_position([0.13, 0.45, 0.22, 0.17])
    axes2[2].set_position([0.375, 0.45, 0.22, 0.17])
    axes2[3].set_position([0.62, 0.45, 0.22, 0.17])
    axes2[4].set_position([0.13, 0.23, 0.22, 0.17])
    axes2[5].set_position([0.375, 0.23, 0.22, 0.17])
    axes2[6].set_position([0.62, 0.23, 0.22, 0.17])

    #fig.patch.set_facecolor([.9, .9, .9])
    #fig.patch.set_edgecolor([.2, .2, .2])
    #fig.patch.set_linewidth(5)

    fig.savefig(os.path.join(path2, 'infographic_{}.svg'.format(var)), dpi=600)
    fig.savefig(os.path.join(path2, 'infographic_{}.png'.format(var)), dpi=600)
    plt.show()